# About this notebook:
- We will be using articles IDs to collect each articles' information, including:
    - Title
    - Abstract
    - Given Mesh Terms
    - Publication Date
- This notebook is part of a series of notebooks under 'Step 2 Articles Collection', where each notebook contain articles published in a particular year.

# Import Libraries and Articles' IDs

In [1]:
## import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from Bio import Entrez
import xml.etree.ElementTree as ET

import time
from tqdm import tqdm

# Set the email address associated with your NCBI account
Entrez.email = "geok1723@gmail.com"

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


df=pd.read_csv("../Step_1_ID_Collection/IDs_2018.csv")

In [2]:
df.head()

,Date_str,ID,Date,Month,Year
0,2018/01/01,32055158,2018-01-01,1,2018
1,2018/01/04,29297265,2018-01-04,1,2018
2,2018/01/23,29357303,2018-01-23,1,2018
3,2018/01/04,29295270,2018-01-04,1,2018
4,2018/01/18,29339485,2018-01-18,1,2018


# Collecting Articles in groups of 100<br>
> The IDs are split into mini-dataframes containing 100 IDs for scraping. <br>
> Output will contain about 360 mini dataframes of articles that are published in this particular year. <br>
> This process was executed using Kaggle, and resulting dataframes are stored in the respective zip-folder. <br>

In [3]:
# determine the group number for each row
group_num = (df.index // 100) + 1

# map the group number to a group label (G1 to G6)
group_label = 'G' + group_num.astype(str)

# add the new column to the DataFrame
df['sub_group'] = group_label

# print(df)
# print(df['sub_group'].nunique())

df['Date']=pd.to_datetime(df['Date'],yearfirst=True)

# group the DataFrame by "Sub_group"
grouped = df.groupby('sub_group')

# create an empty dictionary to store the smaller DataFrames
df_dict = {}

# loop over the groups and create smaller DataFrames
for subgroup, subgroup_df in grouped:
    # construct the name of the smaller DataFrame
    year = subgroup_df['Year'].iloc[0]
    month = subgroup_df['Date'].iloc[0].strftime('%m')
    df_name = f"df_{year}_{month}_{subgroup}"
    # add the smaller DataFrame to the dictionary
    df_dict[df_name] = subgroup_df.copy()

df_ls = list(df_dict.keys())

df_ls.sort()

In [7]:
output_df = {}
for grp in tqdm(df_ls):
    _df = df_dict[grp]
    _s = _df[['ID','Date_str']] 

    print(f"Creating Dataframe of articles for {grp}")
    abstracts = []
    titles = []
    ids = []
    MeSH = []
    Pub_Date = []

    for index,row in tqdm(_s.iterrows()):	

        pub_date = row['Date_str']
        pmid = row['ID']


        try:
            handle = Entrez.efetch(db="pubmed", id=pmid, rettype="abstract", retmode="xml")

            xml_data = handle.read()
            handle.close()

            #obtain abstract and title
            root = ET.fromstring(xml_data)
            title = root.find(".//ArticleTitle").text.strip()
            abst = root.findall(".//AbstractText")

            abstract_text = ""
            for abstract in abst:
                abstract_text += abstract.text

            
            #obtain MeSH
            mesh_terms = ['']
            mesh_list = root.findall(".//MeshHeadingList/MeshHeading")
            for mesh in mesh_list:
                descriptor_name = mesh.find(".//DescriptorName").text
                qualifiers = mesh.findall(".//QualifierName")
                qualifier_names = [qualifier.text for qualifier in qualifiers]
                mesh_terms.append((descriptor_name, qualifier_names))
                
            MeSH.append(mesh_terms)            
            ids.append(pmid)
            abstracts.append(abstract_text)
            titles.append(title)
            Pub_Date.append(pub_date)

        except:
            print(f"Error here in ID{pmid}")

    # Store the data in a pandas dataframe
    data = {"ID": ids, "title": titles,"Pub_Date":Pub_Date, "abstract": abstracts, "MeSH_term":MeSH}
    data_df = pd.DataFrame(data)
    df_name = f"ABSTRACT{grp}"
    output_df[df_name] = data_df
    data_df.to_csv(f"{df_name}.csv", index=False)
    print(output_df.keys())

  0%|          | 0/360 [00:00<?, ?it/s]

Creating Dataframe of articles for df_2018_01_G1



0it [00:00, ?it/s]
1it [00:01,  1.13s/it]
2it [00:02,  1.10s/it]
3it [00:04,  1.47s/it]
4it [00:06,  1.66s/it]
5it [00:07,  1.56s/it]
6it [00:08,  1.48s/it]
7it [00:09,  1.36s/it]
8it [00:11,  1.40s/it]
9it [00:12,  1.31s/it]
10it [00:13,  1.31s/it]
11it [00:14,  1.25s/it]
12it [00:16,  1.21s/it]
13it [00:17,  1.17s/it]
14it [00:18,  1.15s/it]
15it [00:19,  1.21s/it]
16it [00:20,  1.19s/it]
17it [00:21,  1.17s/it]
18it [00:23,  1.20s/it]
19it [00:24,  1.17s/it]
20it [00:25,  1.21s/it]
21it [00:26,  1.25s/it]
22it [00:28,  1.27s/it]
23it [00:29,  1.22s/it]
24it [00:30,  1.24s/it]
25it [00:31,  1.20s/it]
26it [00:33,  1.32s/it]
27it [00:34,  1.26s/it]
28it [00:35,  1.28s/it]
29it [00:37,  1.30s/it]
30it [00:38,  1.24s/it]
31it [00:39,  1.20s/it]
32it [00:40,  1.16s/it]
33it [00:41,  1.14s/it]
34it [00:42,  1.23s/it]
35it [00:44,  1.28s/it]
36it [00:45,  1.30s/it]
37it [00:46,  1.28s/it]
38it [00:48,  1.30s/it]
39it [00:49,  1.24s/it]
40it [00:50,  1.25s/it]
41it [00:51,  1.22s/it]
42it 

Error here in ID29339910



43it [00:55,  1.48s/it]
44it [00:56,  1.46s/it]
45it [00:57,  1.43s/it]
46it [00:59,  1.39s/it]
47it [01:00,  1.30s/it]
48it [01:01,  1.23s/it]
49it [01:02,  1.25s/it]
50it [01:04,  1.35s/it]
51it [01:05,  1.35s/it]
52it [01:06,  1.32s/it]
53it [01:08,  1.47s/it]
54it [01:10,  1.49s/it]
55it [01:11,  1.37s/it]
56it [01:12,  1.29s/it]
57it [01:13,  1.35s/it]
58it [01:14,  1.28s/it]
59it [01:16,  1.29s/it]
60it [01:17,  1.24s/it]
61it [01:19,  1.38s/it]
62it [01:21,  1.61s/it]
63it [01:22,  1.45s/it]
64it [01:23,  1.33s/it]
65it [01:24,  1.26s/it]
66it [01:25,  1.26s/it]
67it [01:26,  1.27s/it]
68it [01:28,  1.27s/it]
69it [01:29,  1.37s/it]
70it [01:31,  1.34s/it]
71it [01:32,  1.32s/it]
72it [01:33,  1.31s/it]
73it [01:35,  1.38s/it]
74it [01:37,  1.61s/it]
75it [01:38,  1.58s/it]
76it [01:40,  1.50s/it]

...


13it [01:03, 12.74s/it]
14it [01:05,  9.29s/it]
15it [01:06,  6.81s/it]
16it [01:07,  5.16s/it]
17it [01:09,  4.01s/it]
18it [01:10,  3.20s/it]
19it [01:11,  2.62s/it]
20it [01:12,  2.23s/it]
21it [01:14,  2.02s/it]
22it [01:15,  1.81s/it]
23it [01:17,  1.67s/it]
24it [01:18,  1.57s/it]
25it [01:19,  1.43s/it]
26it [01:20,  1.34s/it]
27it [01:21,  1.26s/it]
28it [01:23,  1.25s/it]
29it [01:24,  1.20s/it]
30it [01:25,  1.22s/it]
31it [01:26,  1.21s/it]
32it [01:27,  1.17s/it]
33it [01:28,  1.15s/it]
34it [01:30,  1.19s/it]
35it [01:31,  1.16s/it]
36it [01:32,  1.14s/it]
37it [01:33,  1.13s/it]
38it [01:34,  1.18s/it]
39it [01:36,  1.25s/it]

Error here in ID30524355



40it [01:37,  1.29s/it]
41it [01:38,  1.23s/it]
42it [01:39,  1.19s/it]
43it [01:40,  1.23s/it]
44it [01:42,  1.27s/it]
45it [01:43,  1.25s/it]
46it [01:44,  1.26s/it]
47it [01:45,  1.20s/it]
48it [01:47,  1.23s/it]
49it [01:48,  1.22s/it]
50it [01:49,  1.34s/it]
51it [01:51,  1.32s/it]
52it [01:52,  1.25s/it]
53it [01:53,  1.28s/it]

Error here in ID30542362



54it [01:54,  1.21s/it]
55it [01:55,  1.17s/it]
56it [01:56,  1.14s/it]
57it [01:57,  1.12s/it]
58it [01:59,  1.18s/it]
59it [02:00,  1.22s/it]
60it [02:02,  1.31s/it]
